In [1]:
import numpy as np
import pandas as pd

In [2]:
N_USERS = 500
N_MOVIES = 1000
N_RELEVANT_MOVIES = 200
MAX_RATING = 5
MIN_RATING = 1

class ratings_data:
    
    def __init__(self, n_users, n_movies, n_relevant_movies, min_rating, max_rating):
        self.n_users = n_users
        self.n_movies = n_movies
        self.n_relevant_movies = n_relevant_movies
        self.min_rating = min_rating
        self.max_rating = max_rating
        
    def make_predictions(self):
        predicted = np.arange(self.n_movies)
        np.random.shuffle(predicted)
        
        return predicted
    
    def make_actual(self):
        actual = np.random.choice(np.arange(self.n_movies), self.n_relevant_movies)
        
        return actual
    
    def give_ratings(self):
        return np.random.choice(np.arange(self.min_rating, self.max_rating+1), self.n_relevant_movies)
    
    def make_data(self):
        actuals = np.array(list(map(lambda x: dataset.make_actual(), range(self.n_users))))
        predictions = np.array(list(map(lambda x: dataset.make_predictions(), range(self.n_users))))
        ratings = np.array(list(map(lambda x: dataset.give_ratings(), range(self.n_users))))
        return actuals, predictions, ratings
    
    
dataset = ratings_data(N_USERS, N_MOVIES, N_RELEVANT_MOVIES, MIN_RATING, MAX_RATING)

In [3]:
gt, predictions, ratings = dataset.make_data()

gt.shape, predictions.shape, ratings.shape

((500, 200), (500, 1000), (500, 200))

### Order unaware metrics:

1. Hit@K
2. Precision@K
3. Recall@K
4. F1@K

In [4]:
def hit_k(act, pred, k):
    act_set = set(act)
    pred_set = set(pred[:k])
    
    common_set = act_set & pred_set
    
    if (len(common_set)>0):
        return 1
    else:
        return 0
    
def precision_k(act, pred, k):
    act_set = set(act)
    pred_set = set(pred[:k])
    
    common_set = act_set & pred_set
    
    if len(pred_set)==0:
        return 0
    else:
        return len(common_set) / len(pred_set)

def recall_k(act, pred, k):
    act_set = set(act)
    pred_set = set(pred[:k])
    
    common_set = act_set & pred_set
    
    if len(act_set)==0:
        return 0
    else:
        return len(common_set) / len(act_set)

In [5]:
#example

actual = [['A', 'B', 'X'], ['A', 'B', 'Y']]

predicted = [['X', 'Y', 'Z'], ['X', 'Y', 'Z']]

np.mean(list(map(lambda x,y : precision_k(x, y, len(y)), actual, predicted)))

0.3333333333333333

In [6]:
K = [10,20,50,100,200,500]

hits={}
prec={}
rec={}

for k in K:
    hits[k] = np.mean(list(map(lambda x, y: hit_k(x, y, k), gt, predictions)))
    prec[k] = np.mean(list(map(lambda x, y: precision_k(x, y, k), gt, predictions)))
    rec[k] = np.mean(list(map(lambda x, y: recall_k(x, y, k), gt, predictions)))

In [7]:
result = pd.DataFrame(index=K, columns=['Hit_Ratio@K','Precision@K','Recall@K'])

result['Hit_Ratio@K'] = hits.values()
result['Precision@K'] = prec.values()
result['Recall@K'] = rec.values()
result['F1@K'] = 2 * result['Precision@K'] * result['Recall@K'] / (result['Precision@K'] + result['Recall@K'])

result

,Hit_Ratio@K,Precision@K,Recall@K,F1@K
10,0.858,0.18120,0.009987,0.018931
20,0.988,0.18060,0.019905,0.035858
50,1.000,0.17836,0.049133,0.077043
100,1.000,0.17998,0.099172,0.127880
200,1.000,0.17939,0.197725,0.188112
500,1.000,0.18086,0.498308,0.265395


### Order aware metrics:

1. NDCG@K
2. MRR
3. MAP

In [8]:
def dcg_k(act, pred, rating, k):
    act_set = set(act)
    pred_set = set(pred[:k])
    
    common_set = act_set & pred_set    
    
    movie_rating_map = dict(zip(act, rating))
    
    return sum(map(lambda x: movie_rating_map[x] / np.log2(np.where(pred[:k]==x)[0][0]+2), list(common_set)))

def idcg_k(rating, k):
    rating.sort()
    
    return sum(map(lambda x, y: x / np.log2(y+2), rating[::-1][:k], range(k)))

def ndcg_k(act, pred, rating, k):
    return dcg_k(act, pred, rating, k) / idcg_k(rating, k)

In [9]:
ndcg = {}

for k in K:
    ndcg[k] = np.mean(list(map(lambda x, y, z: ndcg_k(x, y, z, k), gt, predictions, ratings)))
    
result['NDCG@K'] = ndcg.values()
result

,Hit_Ratio@K,Precision@K,Recall@K,F1@K,NDCG@K
10,0.858,0.18120,0.009987,0.018931,0.108147
20,0.988,0.18060,0.019905,0.035858,0.111873
50,1.000,0.17836,0.049133,0.077043,0.113986
100,1.000,0.17998,0.099172,0.127880,0.127026
200,1.000,0.17939,0.197725,0.188112,0.164152
500,1.000,0.18086,0.498308,0.265395,0.335024


In [10]:
def mean_reciprocal_rank(gt, predictions):

    def find_reciprocal_rank(act, pred):
        for p in pred:
            if (p in act):
                rank = np.where(pred==p)[0][0] + 1
                return 1/rank

    return np.mean(list(map(lambda x, y: find_reciprocal_rank(x, y), gt, predictions)))

mean_reciprocal_rank(gt, predictions)

0.38042035807890234

In [11]:
def mean_average_precision(gt, predictions):

    def average_precision(act, pred):
        return np.mean(list(map(lambda k: (act[k] == pred[k]) * precision_k(act, pred, k), range(len(act)))))
    
    return np.mean(list(map(lambda x,y: average_precision(x,y), gt, predictions)))

mean_average_precision(gt, predictions)

0.00018382340095863015